## AI Gateway Controlled MCP Servers With Foundry Agents

![image](./images/image.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
mcp_server_name = os.getenv("MCP_SERVER_NAME")
mcp_server_url = os.getenv("MCP_SERVER_URL")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Getting the MCP Server Connection ID

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == mcp_server_name:
        connection_id = connection.id
        break

print(f"The MCP Server Connection ID is: {connection_id}")

### Creating the MCP Tool Spec

In [ ]:
tool = MCPTool(
    server_label = "udemy_demo_mcp_server",
    server_url="",
    require_approval="never",
    project_connection_id=connection_id
)

### Creating the MCP Agent

In [ ]:
agent_name = "hosted-mcp-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are an AI agent that can interact with the MCP server to perform various tasks as requested by the user.",
        tools=[tool],
    )
)

print(f"Created MCP Agent with ID: {agent.id}")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat with the Agent

#### Some Queries to Try Out
- "Tell me something about the recipe with ID 12"
- "What are the ingredients for the recipe with ID 5?"
- "Tell me about the course with ID 3 for author Kuljot Singh Bakshi on Udemy."
- "What is the duration of the course with ID 7 on Udemy?"

In [ ]:
user_query = "Tell me something about the recipe with ID 12"

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": agent_name,
            "type": "agent_reference"
        }
    },
    input=user_query
)

print(f"Agent Response: {response.output_text}")